<a href="https://colab.research.google.com/github/eshauchuk/Bandcamp_fan_counter/blob/main/_14_IMDB_review_custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMDB movie review: description to genre

Google drive mounted automatically. 

In [85]:
import pandas as pd
import numpy as np
import re
import os
from tqdm.notebook import tqdm
tqdm.pandas()

### prepare the data

In [2]:
!pip install tweet-preprocessor
import preprocessor as p

def clean_text(text):
  text = text.replace("#","")
  return p.clean(text)

In [86]:
path = '/content/drive/MyDrive/Skillbox/practice_data/Genre Classification Dataset/'

# read data
dev_data = pd.read_csv(path+'test_data_solution.txt', sep=":::", header=None)
train_data = pd.read_csv(path+'train_data.txt', sep=":::", header=None)
dev_data.columns = ['id', 'movie', 'genre','description']
train_data.columns = ['id', 'movie', 'genre','description']

# changing genre from str to int (cat codes)
train_data['genre'] = (train_data['genre'].astype('category')).cat.codes
dev_data['genre'] = (dev_data['genre'].astype('category')).cat.codes

train_data.head()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,id,movie,genre,description
0,1,Oscar et la dame rose (2009),8,Listening in to a conversation between his do...
1,2,Cupid (1997),24,A brother and sister with a past incestuous r...
2,3,"Young, Wild and Wonderful (1980)",1,As the bus empties the students for their fie...
3,4,The Secret Sin (1915),8,To help their unemployed father make ends mee...
4,5,The Unrecovered (2007),8,The film's title refers not only to the un-re...


In [87]:
# making dev and test datasets out of test_data_solutions
# format source https://simpletransformers.ai/docs/classification-data-formats/
dev_data['labels'] = dev_data['genre']
dev_data['text'] = dev_data['description'].astype(str).progress_map(clean_text)

train_data['labels'] = train_data['genre']
train_data['text'] = train_data['description'].astype(str).progress_map(clean_text)

eval_data = dev_data[['text', 'labels']]
train_data = train_data[['text', 'labels']]

# from sklearn.model_selection import train_test_split
# eval_data, df_test = train_test_split(eval_data, test_size=0.2)

  0%|          | 0/54200 [00:00<?, ?it/s]

  0%|          | 0/54214 [00:00<?, ?it/s]

In [88]:
text_all = pd.concat([train_data['text'], eval_data['text']])

# count all words in the train dataset for the initial vocabsize
results = set()
text_all.str.lower().str.split().apply(results.update)
VOCAB_SIZE = len(results)

word_index = {word:(i+3) for i, word in enumerate(results)}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNKNOWN>"] = 2

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

# последовательность индексов в текст
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

# текст в последовательность индексов
def encode_review(text):
    words = text.lower().split()
    words = ['<START>'] + words
    idxs = [word_index.get(word, word_index['<UNKNOWN>']) for word in words]
    return idxs


In [89]:
# encode a text using those functions
train_data['tokentext'] = train_data.text.apply(encode_review)
eval_data['tokentext'] = eval_data.text.apply(encode_review)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [90]:
MAX_SEQ_LEN = 256 # Финальная длина последовательности

train_data.tokentext = tf.keras.preprocessing.sequence.pad_sequences(
    train_data.tokentext,
    value=word_index["<PAD>"],
    padding='post',
    maxlen=MAX_SEQ_LEN)

eval_data.tokentext = tf.keras.preprocessing.sequence.pad_sequences(
    eval_data.tokentext,
    value=word_index["<PAD>"],
    padding='post',
    maxlen=MAX_SEQ_LEN)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [91]:
eval_data.tokentext[0]

1

### Make model and train


In [35]:
import tensorflow as tf
# setting recurrent_dropout = 0 since only then LSTM will use cuDNN kernels 
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 16),
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(16, return_sequences=True, dropout=0.5, recurrent_dropout=0)),
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(16, return_sequences=False, dropout=0.5, recurrent_dropout=0)),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid),
])


In [15]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [ ]:
BATCH_SIZE = 512
NUM_EPOCHS = 20

history = model.fit(train_data.text,
                    train_data.labels,
                    epochs=NUM_EPOCHS,
                    batch_size=BATCH_SIZE,
                    validation_data=(eval_data.text, eval_data.labels),
                    verbose=1)

In [ ]:
results = model.evaluate(df_test.text, df_test.labels)


print('Test loss: {:.4f}'.format(results[0]))
print('Test accuracy: {:.2f} %'.format(results[1]*100))